# From Simple Linear Regression to Multi-level Models

**Example:  Major League Baseball Player Batting Ability**

- Stan Case Study: [Hierarchical Partial Pooling for Repeated Binary Trials](https://mc-stan.org/users/documentation/case-studies/pool-binary-trials.html)"

- Data: batting records for Major League Baseball players in 1975

- For each player, observe number of hits in first 45 at-bats.

- Estimate probability of hit per at-bat.

- Compare predicted success with actual outcomes during the rest of the season.

- Three models:  complete pooling; no pooling; some pooling

In [ ]:
# Use CmdStanPy, also numpy and pandas
import numpy as np
import pandas as pd

from cmdstanpy import cmdstan_path, CmdStanModel
print('using CmdStan: {}'.format(cmdstan_path()))

In [ ]:
#  Data: batting records for Major League Baseball players in 1975
season_1975 = pd.read_csv('efron-morris-75-data.csv')
season_1975.iloc[0:5,0:5]

In [ ]:
import statistics
players = [' '.join([season_1975.iloc[x,0],season_1975.iloc[x,1]]) for x in range(season_1975.shape[0])]
mapping = dict(zip(range(len(players)), players))
season_1975.rename(index=mapping, inplace=True)
plt = season_1975['BattingAverage'].plot.bar()
plt.axhline(linewidth=2, color='g', y=statistics.mean(season_1975['BattingAverage']))


In [ ]:
#  Extract relevant columns into dictionary of inputs:  'N', 'K', 'y'
data_dict = {'N': season_1975.shape[0], 'y' : season_1975['Hits'].tolist(), 'K' : season_1975['At-Bats'].tolist()}
data_dict

#### Binomial distribution n=45, p=.400

In [ ]:
# expected number of hits in quantiles 1:99 of binomial pmf
from scipy.stats import binom
import matplotlib.pyplot as plt
## plug in stats for Roberto Clemente
n = 45
p = 0.4
x = np.arange(binom.ppf(0.01, n, p), binom.ppf(0.99, n, p))

# plot with matplotlib
fig, ax = plt.subplots(1, 1)
rv = binom(n, p)
ax.plot(x, binom.pmf(x, n, p), 'bo', ms=8, label='binom pmf')
ax.vlines(x, 0, rv.pmf(x), colors='k', linestyles='-', lw=1,
          label='Expected hits in 45-at bats, batting average .400')
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width, box.height*0.8])
ax.legend(loc='lower center', bbox_to_anchor=(0.5, 1.05),
          fancybox=True, shadow=True)

p = plt.show()
p

### Model 1:  Complete Pooling - `simple_pool.stan`

All players are the same; single estimate of player ability.

In [ ]:
model_complete_pool = CmdStanModel(stan_file='simple_pool.stan')
print(model_complete_pool.code())

model_complete_pool.compile()
complete_pool_fit = model_complete_pool.sample(data=data_dict)
complete_pool_fit.summary().round(decimals=2)

### Model 2:  No Pooling - `simple_no_pool.stan`

Every player is different - examine estimates for top 5 players in the dataset.

In [ ]:
model_no_pool = CmdStanModel(stan_file='simple_no_pool.stan')
print(model_no_pool.code())

model_no_pool.compile()
no_pool_fit = model_no_pool.sample(data=data_dict)
no_pool_fit.summary().round(decimals=2).iloc[0:5,:]

### Model 3:  Partial Pooling - `simple_hier.stan`

There is a general population of players, estimate group-level ability together with individual player ability.

In [ ]:
model_hier = CmdStanModel(stan_file='simple_hier.stan')
print(model_hier.code())

model_hier.compile()
hier_fit = model_hier.sample(data=data_dict, adapt_delta=0.95)
hier_fit.summary().round(decimals=2).iloc[0:7,:]

In [ ]:
hier_fit.diagnose()

In [ ]:
# Visualization

# Get thetas from no pooling model
players_summary_no_pool = no_pool_fit.summary().iloc[1:,0]
players_summary_no_pool.index = players

# Get thetas from partial pooling  model
players_summary_hier = hier_fit.summary().iloc[3:,0]
players_summary_hier.index = players

players_summary = pd.DataFrame(dict(no_pooling = players_summary_no_pool, partial_pooling = players_summary_hier))

# Plot
plt = players_summary.plot.bar()
# Add complete pooling estimate
plt.axhline(linewidth=2, color='g', y=0.27)